# Extract

In [1]:
import requests
from datetime import date
import json

api_key = "TTKOHGL05DM097G0"
companies = ["AAPL", "GOOG", "MSFT"]

for company in companies:
  response = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={company}&interval=5min&apikey={api_key}")
  # with open(f"raw_data/{company}_{date.today()}.json", "w") as file:
  #   json.dump(response.json(), file)

# Transform

In [2]:
import pandas as pd


df = pd.read_json("raw_data/before_pydantic/AAPL_2025-10-02.json")
df.head() 

,Meta Data,Time Series (5min)
1. Information,"Intraday (5min) open, high, low, close prices ...",NaN
2. Symbol,AAPL,NaN
3. Last Refreshed,2025-10-01 19:55:00,NaN
4. Interval,5min,NaN
5. Output Size,Compact,NaN


In [3]:
df = df[6:].reset_index()


In [4]:
df

,index,Meta Data,Time Series (5min)
0,2025-10-01 19:55:00,NaN,"{'1. open': '255.2500', '2. high': '255.2900',..."
1,2025-10-01 19:50:00,NaN,"{'1. open': '255.2500', '2. high': '255.3000',..."
2,2025-10-01 19:45:00,NaN,"{'1. open': '255.2343', '2. high': '255.3000',..."
3,2025-10-01 19:40:00,NaN,"{'1. open': '255.2200', '2. high': '255.3000',..."
4,2025-10-01 19:35:00,NaN,"{'1. open': '255.2800', '2. high': '255.3000',..."
...,...,...,...
95,2025-10-01 12:00:00,NaN,"{'1. open': '256.1000', '2. high': '256.4700',..."
96,2025-10-01 11:55:00,NaN,"{'1. open': '255.7900', '2. high': '256.1200',..."
97,2025-10-01 11:50:00,NaN,"{'1. open': '255.6856', '2. high': '255.7900',..."
98,2025-10-01 11:45:00,NaN,"{'1. open': '255.7950', '2. high': '256.0400',..."


In [5]:
cleaned_df = pd.json_normalize(df["Time Series (5min)"])
cleaned_df["date"] = df["index"]

In [6]:
cleaned_df.columns

Index(['1. open', '2. high', '3. low', '4. close', '5. volume', 'date'], dtype='object')

In [7]:
cleaned_df.rename(columns={
    "1. open": "open", 
    "2. high": "high", 
    "3. low": "low", 
    "4. close": "close", 
    "5. volume": "volume"
}, inplace=True)

In [8]:
cleaned_df = cleaned_df[["date", "open", "high", "low", "close", "volume"]]

In [9]:
cleaned_df

,date,open,high,low,close,volume
0,2025-10-01 19:55:00,255.2500,255.2900,255.2300,255.2800,1939
1,2025-10-01 19:50:00,255.2500,255.3000,255.2400,255.2585,1940
2,2025-10-01 19:45:00,255.2343,255.3000,255.2300,255.2500,2646
3,2025-10-01 19:40:00,255.2200,255.3000,255.2200,255.2500,2351
4,2025-10-01 19:35:00,255.2800,255.3000,255.2200,255.2600,2345
...,...,...,...,...,...,...
95,2025-10-01 12:00:00,256.1000,256.4700,256.0900,256.3600,316957
96,2025-10-01 11:55:00,255.7900,256.1200,255.7200,256.1100,308784
97,2025-10-01 11:50:00,255.6856,255.7900,255.5500,255.7900,242616
98,2025-10-01 11:45:00,255.7950,256.0400,255.6200,255.6850,302899


In [10]:
cleaned_df.dtypes

date      object
open      object
high      object
low       object
close     object
volume    object
dtype: object

In [11]:
cleaned_df["date"] = pd.to_datetime(cleaned_df["date"])
cleaned_df[["open", "high", "low", "close"]] = cleaned_df[["open", "high", "low", "close"]].astype("float")
cleaned_df["volume"] = cleaned_df["volume"].astype("int")

In [12]:
cleaned_df.dtypes

date      datetime64[ns]
open             float64
high             float64
low              float64
close            float64
volume             int32
dtype: object

In [13]:
cleaned_df["daily_change_percentage"] = ((cleaned_df["close"] - cleaned_df["open"]) / cleaned_df["open"]) * 100

In [14]:
cleaned_df

,date,open,high,low,close,volume,daily_change_percentage
0,2025-10-01 19:55:00,255.2500,255.29,255.23,255.2800,1939,0.011753
1,2025-10-01 19:50:00,255.2500,255.30,255.24,255.2585,1940,0.003330
2,2025-10-01 19:45:00,255.2343,255.30,255.23,255.2500,2646,0.006151
3,2025-10-01 19:40:00,255.2200,255.30,255.22,255.2500,2351,0.011755
4,2025-10-01 19:35:00,255.2800,255.30,255.22,255.2600,2345,-0.007835
...,...,...,...,...,...,...,...
95,2025-10-01 12:00:00,256.1000,256.47,256.09,256.3600,316957,0.101523
96,2025-10-01 11:55:00,255.7900,256.12,255.72,256.1100,308784,0.125103
97,2025-10-01 11:50:00,255.6856,255.79,255.55,255.7900,242616,0.040831
98,2025-10-01 11:45:00,255.7950,256.04,255.62,255.6850,302899,-0.043003


In [15]:
import pandas as pd
import os

def transform(file):
    df = pd.read_json(f"raw_data/before_pydantic/{file}")[6:].reset_index()

    cleaned_df = pd.json_normalize(df["Time Series (5min)"])
    cleaned_df["date"] = df["index"]
    cleaned_df.rename(columns={
        "1. open": "open", 
        "2. high": "high", 
        "3. low": "low", 
        "4. close": "close", 
        "5. volume": "volume"
    }, inplace=True)
    cleaned_df = cleaned_df[["date", "open", "high", "low", "close", "volume"]]


    cleaned_df["date"] = pd.to_datetime(cleaned_df["date"])
    cleaned_df[["open", "high", "low", "close"]] = cleaned_df[["open", "high", "low", "close"]].astype("float")
    cleaned_df["volume"] = cleaned_df["volume"].astype("int")


    cleaned_df["daily_change_percentage"] = ((cleaned_df["close"] - cleaned_df["open"]) / cleaned_df["open"]) * 100

    os.makedirs("cleaned_data", exist_ok=True)
    cleaned_df.to_json(f"./cleaned_data/{file}")

    return cleaned_df

In [16]:
for file in os.listdir("./raw_data/before_pydantic"):
    transform(file)

# Load

In [17]:
import sqlite3

conn = sqlite3.connect("./db/stocks.db")
cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS stock_daily_data (
        symbol TEXT NOT NULL,
        date DATE NOT NULL,
        open_price REAL,
        high_price REAL,
        low_price REAL,
        close_price REAL,
        volume INTEGER,
        daily_change_percentage REAL,
        extraction_timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        PRIMARY KEY (symbol, date)
    );
""")

conn.commit()
conn.close()

In [18]:
df = transform("MSFT_2025-10-02.json")

df

,date,open,high,low,close,volume,daily_change_percentage
0,2025-10-01 19:55:00,519.215,519.7000,519.1300,519.650,1368,0.083780
1,2025-10-01 19:50:00,519.200,519.3000,519.1000,519.300,289,0.019260
2,2025-10-01 19:45:00,518.870,519.3000,518.8700,519.200,771,0.063600
3,2025-10-01 19:40:00,518.870,519.3000,518.8700,519.200,1917,0.063600
4,2025-10-01 19:35:00,518.985,519.1000,518.8700,518.985,637,0.000000
...,...,...,...,...,...,...,...
95,2025-10-01 12:00:00,513.910,514.2700,513.8100,514.140,193798,0.044755
96,2025-10-01 11:55:00,514.070,514.3200,513.6500,513.905,110131,-0.032097
97,2025-10-01 11:50:00,514.080,514.4225,514.0200,514.020,105678,-0.011671
98,2025-10-01 11:45:00,514.390,514.4800,513.9100,514.075,132044,-0.061238


In [19]:
df["symbol"] = "MSFT"

In [20]:
df

,date,open,high,low,close,volume,daily_change_percentage,symbol
0,2025-10-01 19:55:00,519.215,519.7000,519.1300,519.650,1368,0.083780,MSFT
1,2025-10-01 19:50:00,519.200,519.3000,519.1000,519.300,289,0.019260,MSFT
2,2025-10-01 19:45:00,518.870,519.3000,518.8700,519.200,771,0.063600,MSFT
3,2025-10-01 19:40:00,518.870,519.3000,518.8700,519.200,1917,0.063600,MSFT
4,2025-10-01 19:35:00,518.985,519.1000,518.8700,518.985,637,0.000000,MSFT
...,...,...,...,...,...,...,...,...
95,2025-10-01 12:00:00,513.910,514.2700,513.8100,514.140,193798,0.044755,MSFT
96,2025-10-01 11:55:00,514.070,514.3200,513.6500,513.905,110131,-0.032097,MSFT
97,2025-10-01 11:50:00,514.080,514.4225,514.0200,514.020,105678,-0.011671,MSFT
98,2025-10-01 11:45:00,514.390,514.4800,513.9100,514.075,132044,-0.061238,MSFT


In [21]:

df['date'] = df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [22]:
df = df[["symbol", "date", "open", "high", "low", "close", "volume", "daily_change_percentage"]]

In [23]:
conn = sqlite3.connect("./db/stocks.db")
cursor = conn.cursor()

sql_command = """
    INSERT INTO stock_daily_data (symbol, date, open_price, high_price, low_price, close_price, volume, daily_change_percentage)
    VALUES (?,?,?,?,?,?,?,?) 
"""

data_into_table = df.values.tolist()

cursor.executemany(sql_command, data_into_table)

conn.commit()
conn.close()

# Final Code Without Pydantic

In [24]:
import pandas as pd
import os
import sqlite3

def transform_load(file):
    # Reading json files
    df = pd.read_json(f"raw_data/{file}")[6:].reset_index()

    # Cleaning and renaming
    cleaned_df = pd.json_normalize(df["Time Series (5min)"])
    cleaned_df["date"] = df["index"]
    cleaned_df.rename(columns={
        "1. open": "open", 
        "2. high": "high", 
        "3. low": "low", 
        "4. close": "close", 
        "5. volume": "volume"
    }, inplace=True)

    # Changing types
    cleaned_df["date"] = pd.to_datetime(cleaned_df["date"])
    cleaned_df[["open", "high", "low", "close"]] = cleaned_df[["open", "high", "low", "close"]].astype("float")
    cleaned_df["volume"] = cleaned_df["volume"].astype("int")

    # Changing timestamp to add into sqlite database
    cleaned_df['date'] = cleaned_df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    # adding new columns
    cleaned_df["daily_change_percentage"] = ((cleaned_df["close"] - cleaned_df["open"]) / cleaned_df["open"]) * 100
    cleaned_df["symbol"] = file.split("_")[0] # symbol column for database

    # Changing order of the columns
    cleaned_df = cleaned_df[["symbol", "date", "open", "high", "low", "close", "volume", "daily_change_percentage"]]

    # Saving cleaned data into json files
    os.makedirs("cleaned_data", exist_ok=True)
    cleaned_df.to_json(f"./cleaned_data/{file}")


    # create Sqlite database
    conn = sqlite3.connect("./db/stocks.db")
    cursor = conn.cursor()

    # Primary key is symbol and date to "Make sure no duplicate records are inserted for the same stock/date"
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS stock_daily_data (
            symbol TEXT NOT NULL,
            date DATE NOT NULL,
            open_price REAL,
            high_price REAL,
            low_price REAL,
            close_price REAL,
            volume INTEGER,
            daily_change_percentage REAL,
            extraction_timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            PRIMARY KEY (symbol, date)
        );
    """)

    sql_command = """
        INSERT INTO stock_daily_data (symbol, date, open_price, high_price, low_price, close_price, volume, daily_change_percentage)
        VALUES (?,?,?,?,?,?,?,?) 
    """
    data_into_table = cleaned_df.values.tolist()

    cursor.executemany(sql_command, data_into_table)

    conn.commit()
    conn.close()


    return cleaned_df

In [25]:
for file in os.listdir("./raw_data/before_pydantic"):
    transform(file)

# Pydantic Test

In [26]:
%pip install pydantic[root]

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pydantic[root] in c:\users\andre\anaconda3\lib\site-packages (2.11.9)



In [27]:
%pip install pydantic[email]

Note: you may need to restart the kernel to use updated packages.


In [28]:
data = {
    "2025-10-01T10:00:00": {
        "temperature": "22.5",
        "humidity": "60"
    },
    "2025-10-01T11:00:00": {
        "temperature": "23.1",
        "humidity": "58"
    }
}

In [29]:
from pydantic import BaseModel, RootModel, EmailStr
from typing import Dict

class Measurement(BaseModel):
    temperature: float
    humidity: int

class SensorResponse(RootModel[Dict[str, Measurement]]):
    pass

In [30]:
p = SensorResponse.model_validate(data)

# print(p)
print(p)

root={'2025-10-01T10:00:00': Measurement(temperature=22.5, humidity=60), '2025-10-01T11:00:00': Measurement(temperature=23.1, humidity=58)}


In [31]:
from pydantic import BaseModel, RootModel, Field
from typing import Dict

class Observation(BaseModel):
    open: float = Field(alias="1. open")
    high: float = Field(alias="2. high")
    low: float = Field(alias="3. low")
    close: float = Field(alias="4. close")
    volume: int = Field(alias="5. volume")

class TimeSeries(RootModel[Dict[str, Observation]]):
    pass

class MetaDataIntraday(BaseModel):
    Information: str = Field(alias="1. Information")
    Symbol: str = Field(alias="2. Symbol")
    Last_Refreshed: str = Field(alias="3. Last Refreshed")
    Interval: str = Field(alias="4. Interval")
    Output_Size: str = Field(alias="5. Output Size")
    Time_Zone: str = Field(alias="6. Time Zone")

class ApiResponse(BaseModel):
    meta_data: MetaDataIntraday = Field(alias="Meta Data")
    time_series: TimeSeries = Field(alias="Time Series (5min)")

In [42]:
import requests

api_key = "PY21HHFY6FPLE9Y3"
url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=AAPL&interval=5min&apikey={api_key}"
response = requests.get(url).json()

try:
    validated = ApiResponse.model_validate(response)
    print(validated.time_series.model_dump())
except Exception as e:
    print("Error:", e)


{'2025-10-03 19:55:00': {'open': 257.86, 'high': 257.9, 'low': 257.85, 'close': 257.9, 'volume': 1600}, '2025-10-03 19:50:00': {'open': 257.86, 'high': 257.9, 'low': 257.85, 'close': 257.85, 'volume': 2393}, '2025-10-03 19:45:00': {'open': 257.85, 'high': 257.9, 'low': 257.85, 'close': 257.85, 'volume': 1533}, '2025-10-03 19:40:00': {'open': 257.85, 'high': 257.9, 'low': 257.85, 'close': 257.85, 'volume': 799}, '2025-10-03 19:35:00': {'open': 257.8045, 'high': 257.9788, 'low': 257.8, 'close': 257.84, 'volume': 138}, '2025-10-03 19:30:00': {'open': 257.77, 'high': 257.91, 'low': 257.77, 'close': 257.9, 'volume': 1794}, '2025-10-03 19:25:00': {'open': 257.92, 'high': 257.97, 'low': 257.71, 'close': 257.97, 'volume': 3096}, '2025-10-03 19:20:00': {'open': 257.95, 'high': 257.9799, 'low': 257.91, 'close': 257.93, 'volume': 387}, '2025-10-03 19:15:00': {'open': 257.94, 'high': 257.99, 'low': 257.91, 'close': 257.95, 'volume': 532}, '2025-10-03 19:10:00': {'open': 257.97, 'high': 257.99, 'lo

In [63]:
pd.DataFrame(validated.time_series.model_dump()).T.reset_index().rename({"index": "date"})

,index,open,high,low,close,volume
0,2025-10-03 19:55:00,516.6700,517.00,516.670,516.9500,687.0
1,2025-10-03 19:50:00,516.8200,517.00,516.650,516.8350,766.0
2,2025-10-03 19:45:00,516.8900,517.00,516.600,516.8200,1101.0
3,2025-10-03 19:40:00,516.9900,517.00,516.670,516.8800,232.0
4,2025-10-03 19:35:00,516.8000,517.00,516.670,517.0000,217.0
...,...,...,...,...,...,...
95,2025-10-03 12:00:00,519.6921,520.13,519.690,520.0300,113735.0
96,2025-10-03 11:55:00,519.7600,519.88,519.550,519.6450,73446.0
97,2025-10-03 11:50:00,519.8100,520.03,519.680,519.7835,99089.0
98,2025-10-03 11:45:00,519.5900,519.81,519.415,519.8100,69124.0


In [ ]:
import requests
from datetime import date
import json

api_key = "TTKOHGL05DM097G0"
companies = ["AAPL", "GOOG", "MSFT"]

for company in companies:
  response = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={company}&interval=5min&apikey={api_key}").json()
  
  validated = ApiResponse.model_validate(response)
  df_from_api = pd.DataFrame(validated.time_series.model_dump()).T
  df_from_api.reset_index(inplace=True)
  df_from_api.rename(columns={"index": "date"}, inplace=True)

  result = df_from_api

In [ ]:
result.renam

,index,open,high,low,close,volume
0,2025-10-03 19:55:00,516.6700,517.00,516.670,516.9500,687.0
1,2025-10-03 19:50:00,516.8200,517.00,516.650,516.8350,766.0
2,2025-10-03 19:45:00,516.8900,517.00,516.600,516.8200,1101.0
3,2025-10-03 19:40:00,516.9900,517.00,516.670,516.8800,232.0
4,2025-10-03 19:35:00,516.8000,517.00,516.670,517.0000,217.0
...,...,...,...,...,...,...
95,2025-10-03 12:00:00,519.6921,520.13,519.690,520.0300,113735.0
96,2025-10-03 11:55:00,519.7600,519.88,519.550,519.6450,73446.0
97,2025-10-03 11:50:00,519.8100,520.03,519.680,519.7835,99089.0
98,2025-10-03 11:45:00,519.5900,519.81,519.415,519.8100,69124.0


# Final Code With Pydantic

In [ ]:
def transform(file):
    # Reading json files
    df = pd.read_json(f"raw_data/{file}")

    # adding new columns
    df["daily_change_percentage"] = ((df["close"] - df["open"]) / df["open"]) * 100
    df["symbol"] = file.split("_")[0]

    # Changing timestamp to add it into sqlite database
    df['date'] = df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')

    # Changing order of the columns
    df = df[["symbol", "date", "open", "high", "low", "close", "volume", "daily_change_percentage"]]

    print(df)

In [76]:
for file in os.listdir("./raw_data"):
    transform(file)

   symbol                 date      open      high       low     close  \
0    AAPL  2025-10-03 19:55:00  257.8600  257.9000  257.8500  257.9000   
1    AAPL  2025-10-03 19:50:00  257.8600  257.9000  257.8500  257.8500   
2    AAPL  2025-10-03 19:45:00  257.8500  257.9000  257.8500  257.8500   
3    AAPL  2025-10-03 19:40:00  257.8500  257.9000  257.8500  257.8500   
4    AAPL  2025-10-03 19:35:00  257.8045  257.9788  257.8000  257.8400   
..    ...                  ...       ...       ...       ...       ...   
95   AAPL  2025-10-03 12:00:00  258.4131  258.4800  258.2605  258.2605   
96   AAPL  2025-10-03 11:55:00  258.4300  258.4730  258.2600  258.4050   
97   AAPL  2025-10-03 11:50:00  258.4700  258.6000  258.3100  258.4300   
98   AAPL  2025-10-03 11:45:00  258.7994  258.8400  258.3000  258.4650   
99   AAPL  2025-10-03 11:40:00  258.8400  259.2400  258.5700  258.8000   

     volume  daily_change_percentage  
0      1600                 0.015512  
1      2393                -0.003